In [1]:
import os
import timeit
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ModelCheckpoint

import models
import tools as tl
import tensorflow as tf
import image_process as ip
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
import newgenerator
from keras_bcnn.models import BayesianUNet2D
from tensorflow.keras.backend import categorical_crossentropy,binary_crossentropy
from scipy.stats import entropy, norm
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Lambda,Reshape,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
import gc
from matplotlib import pylab as plt

Using TensorFlow backend.


In [2]:
#変数設定
method = 'unet_patch'
data = "ips"
model_name = "unet"
method = 'unet_patch'
patch_size = 160
batch_size = 12
output_channel = 3
mc_iteration=10
reflect = False
n_class=3

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    #Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def cce_dice_loss(y_true, y_pred):
    loss = categorical_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss
  
def original(y_true, y_pred):#重要　出力のエントロピーを0に近づけるloss
    zero=tf.zeros([1,160,160])
    mse = tf.keras.losses.MeanSquaredError()
    e=-1*(y_pred[:,:,:,0]*tf.math.log(y_pred[:,:,:,0])+y_pred[:,:,:,1]*tf.math.log(y_pred[:,:,:,1])+y_pred[:,:,:,2]*tf.math.log(y_pred[:,:,:,2]))
    e=tf.where(tf.math.is_nan(e), tf.zeros_like(e), e)
    return mse(zero, e)

def original2(y_true, y_pred):
    tmp = tf.math.argmax(y_true, axis=3)
    tmp2 = tf.math.argmax(y_pred, axis=3)
    tmp3 = tf.math.subtract(tmp,tmp2)
    tmp4 = tf.where(tmp3!=0,1.,0.)
    e=-1*(y_pred[:,:,:,0]*tf.math.log(y_pred[:,:,:,0])+y_pred[:,:,:,1]*tf.math.log(y_pred[:,:,:,1])+y_pred[:,:,:,2]*tf.math.log(y_pred[:,:,:,2]))
    e=tf.where(tf.math.is_nan(e), tf.zeros_like(e), e)
    e = tf.clip_by_value(e, clip_value_max=1, clip_value_min=0)
    #return categorical_crossentropy(y_true, y_pred) +dice_loss(tmp4, e)
    return categorical_crossentropy(y_true, y_pred) +binary_crossentropy(tmp4, e)

for mode in ["test"] :#"train","val" 
    for data_num in [1,2,3,4,5]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        os.makedirs(save_path + '%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"%s"%mode, exist_ok=True)
        os.makedirs(save_path+"label", exist_ok=True)
        os.makedirs(save_path + '%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/probability'%mode, exist_ok=True)

In [4]:
def get_ytrue(mask_array, data):
    #ips画像の赤を1、緑を2、青を3、紫を0にする関数
        if data == "ips":
            height, width, _ = mask_array.shape
            good_label = ((mask_array[:,:,0] == 255)&
                          (mask_array[:,:,1] == 0)&
                          (mask_array[:,:,2] == 0)
                         ) * np.ones((height, width)) * 1
            bad_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 255)&
                         (mask_array[:,:,2] == 0)
                        ) * np.ones((height, width)) * 2
            bgd_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 0)&
                         (mask_array[:,:,2] == 255)
                        ) * np.ones((height, width)) * 3
            y_true = good_label + bad_label + bgd_label
            return y_true
        else:#melanomaの処理
            height, width = mask_array.shape
            y_true = (mask_array == 255) * np.ones((height, width))
            return y_true

In [5]:
def make_prediction_map(size, test_step, width, height, pre_map, ref_num, data, n_class):
    #出力結果、出力結果を255で割った結果(評価に使う)、エントロピーマップ、確率マップを出力
    print("Start visualize image...")
    num = 0
    pre_label = np.zeros((height,width,n_class))
    for y in range(0,height-size+1,test_step):
        for x in range(0,width-size+1,test_step):
            patch = np.ones((size,size,n_class))
            patch = patch*pre_map[num]
            pre_label[y:y+size,x:x+size,:] += patch
            num += 1
    sumaltion=np.sum(pre_label,axis=2)
    pre_label[:,:,0]=pre_label[:,:,0]/sumaltion
    pre_label[:,:,1]=pre_label[:,:,1]/sumaltion
    pre_label[:,:,2]=pre_label[:,:,2]/sumaltion
    e=entropy(pre_label,axis=2)
    max=pre_label.max(axis=2)
    pro = np.zeros((height,width,n_class))
    pro1=pre_label[:,:,0]
    pro2=pre_label[:,:,1]
    pro3=pre_label[:,:,2]
    pro1[pro1<max]=0
    pro2[pro2<max]=0
    pro3[pro3<max]=0
    pro[:,:,0]=pro1
    pro[:,:,1]=pro2
    pro[:,:,2]=pro3
    pro*=255   
    if data in ['ips']:
        vis_img, label_img = tl.get_InfImg(pre_label, data)
    elif data in ['melanoma', 'colonoscopy_tissue']:
        others = pre_label[:,:,1]
        target = pre_label[:,:,0]
        label_img = (target > others)*1
        vis_img = label_img*255
    return vis_img, label_img,e,pro

In [6]:
def convert2label(name, save_path):
    mask = np.array(Image.open(save_path+"/test/" + name + '.png'), dtype=int)
    label = mask / 255
    if data == 'ips':
        label = label[:, :, 0] * 1 + label[:, :, 1] * 2 + label[:, :, 2] * 3
        label = Image.fromarray(np.uint8(label))
        label.save(save_path + '/label/' + name + '.png')
    else:
        label = Image.fromarray(np.uint8(label))
        label.save(save_path + '/label/' + name + '.png')

In [7]:
def sensitivity_specificity(y_true, y_pred):#sensitivity_specificity loss
    l=0.5
    true_positives = K.sum(((y_true - y_pred)**2)*y_pred)
    possible_positives = K.sum(K.clip(y_true, 0, 1))
    
    true_negatives = K.sum(((y_true - y_pred)**2)*(1-y_pred))
    possible_negatives = K.sum(K.clip(1-y_true, 0, 1))
    return l*(true_positives / (possible_positives + K.epsilon()))+(1-l)*(true_negatives / (possible_negatives + K.epsilon()))

In [8]:
#一旦categorical_crossentropyで学習する.
for data_num in [1,2,3,4,5]:
    epochs = 5
    clear_session()
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        for k in range(len(physical_devices)):
            tf.config.experimental.set_memory_growth(physical_devices[k], True)
            print('memory growth:', tf.config.experimental.get_memory_growth(physical_devices[k]))
    else:
        print("Not enough GPU hardware devices available")
    save_path = tl.get_save_path(data, method, model_name,patch_size, data_num)
    model = BayesianUNet2D((patch_size, patch_size, 3), output_channel).build()
    model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    tl.make_dirs(data, method, model_name,patch_size, data_num)
    
    train="/home/sora/new_project/crop/dataset_%d/train/"%data_num
    val="/home/sora/new_project/crop/dataset_%d/val/"%data_num
    # train -----------------------------------------------------------
    train_gen = newgenerator.ImageSequence(train,batch_size,"train",data_num)
    valid_gen = newgenerator.ImageSequence(val, batch_size,"val",data_num)
    print("dataset_%d start!!"%data_num)
    os.makedirs(save_path + 'weights', exist_ok=True)
    model_checkpoint = ModelCheckpoint(
        filepath=os.path.join(save_path,'weights', 'weights.h5'),
        monitor="val_loss",
        verbose=1,
        save_best_only=True)
    history = model.fit_generator(generator=train_gen,
        epochs=epochs,
        steps_per_epoch=len(train_gen),
        verbose=1,
        callbacks=[model_checkpoint],
        validation_data=valid_gen,
        validation_steps=len(valid_gen))

    tl.draw_train_loss_plot(history, save_path)
    del model,history,train_gen,valid_gen,model_checkpoint,save_path
    gc.collect()

memory growth: True
memory growth: True
dataset_1 start!!
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
2485/2485 [==============================] - ETA: 0s - loss: 0.4428 - accuracy: 0.4032

/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: Training option is ignored..
  return py_builtins.overload_of(f)(*args)



Epoch 00001: val_loss improved from inf to 0.36396, saving model to ./ips/unet_patch/unet/size_160/dataset_1/weights/weights.h5
2485/2485 [==============================] - 910s 366ms/step - loss: 0.4428 - accuracy: 0.4032 - val_loss: 0.3640 - val_accuracy: 0.5111
Epoch 2/10
2485/2485 [==============================] - ETA: 0s - loss: 0.4509 - accuracy: 0.4020
Epoch 00002: val_loss did not improve from 0.36396
2485/2485 [==============================] - 287s 116ms/step - loss: 0.4509 - accuracy: 0.4020 - val_loss: 0.3985 - val_accuracy: 0.4699
Epoch 3/10
2485/2485 [==============================] - ETA: 0s - loss: 0.4562 - accuracy: 0.3977
Epoch 00003: val_loss did not improve from 0.36396
2485/2485 [==============================] - 289s 116ms/step - loss: 0.4562 - accuracy: 0.3977 - val_loss: 0.3967 - val_accuracy: 0.4711
Epoch 4/10
2485/2485 [==============================] - ETA: 0s - loss: 0.5006 - accuracy: 0.3765
Epoch 00004: val_loss did not improve from 0.36396
2485/2485 [==

In [8]:
def draw_train_loss_plot2(history, save_path):
    v_l = history.history["val_loss"]
    vv = history.history["val_lambda_2_loss"]
    vvv = history.history["val_lambda_2_1_loss"]
    nb_epoch = len(v_l)
    fig, ax = plt.subplots()
    ax.set_xlabel("epoch")  # x軸ラベル
    ax.set_ylabel("loss")  # y軸ラベル
    # ax.set_aspect('equal') # スケールを揃える
    ax.grid()            # 罫線
    ax.plot(range(nb_epoch), v_l, label="val_loss")
    ax.plot(range(nb_epoch), vv, label="val_CE_loss")
    ax.plot(range(nb_epoch), vvv, label="val_MSE_loss")
    fig.tight_layout()  # レイアウトの設定
    ax.legend(["val_loss","val_CE_loss","val_MSE_loss"],loc='best', fontsize=10)
    plt.savefig(save_path + "loss2.png")
    plt.close()
def draw_train_loss_plot3(history, save_path):
    v_l = history.history["lambda_2_loss"]
    vv = history.history["val_lambda_2_loss"]
    #vvv = history.history["val_lambda_5_1_loss"]
    nb_epoch = len(v_l)
    fig, ax = plt.subplots()
    ax.set_xlabel("epoch")  # x軸ラベル
    ax.set_ylabel("loss")  # y軸ラベル
    # ax.set_aspect('equal') # スケールを揃える
    ax.grid()            # 罫線
    ax.plot(range(nb_epoch), v_l, label="CE_loss")
    ax.plot(range(nb_epoch), vv, label="val_CE_loss")
    #ax.plot(range(nb_epoch), vvv, label="val_MSE_loss")
    fig.tight_layout()  # レイアウトの設定
    ax.legend(["CE_loss","val_CE_loss"],loc='best', fontsize=10)
    plt.savefig(save_path + "loss3.png")
    plt.close()
def draw_train_loss_plot4(history, save_path):
    v_l = history.history["lambda_2_1_loss"]
    vv = history.history["val_lambda_2_1_loss"]
    #vvv = history.history["val_lambda_5_1_loss"]
    nb_epoch = len(v_l)
    fig, ax = plt.subplots()
    ax.set_xlabel("epoch")  # x軸ラベル
    ax.set_ylabel("loss")  # y軸ラベル
    # ax.set_aspect('equal') # スケールを揃える
    ax.grid()            # 罫線
    ax.plot(range(nb_epoch), v_l, label="MSE_loss")
    ax.plot(range(nb_epoch), vv, label="val_NSE_loss")
    #ax.plot(range(nb_epoch), vvv, label="val_MSE_loss")
    fig.tight_layout()  # レイアウトの設定
    ax.legend(["MSE_loss","val_MSE_loss"],loc='best', fontsize=10)
    plt.savefig(save_path + "loss4.png")
    plt.close()

In [9]:
#自作lossで学習
for data_num in [2,3,4,5]:
    clear_session()
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        for k in range(len(physical_devices)):
            tf.config.experimental.set_memory_growth(physical_devices[k], True)
            print('memory growth:', tf.config.experimental.get_memory_growth(physical_devices[k]))
    else:
        print("Not enough GPU hardware devices available")
        
    save_path = tl.get_save_path(data, method, model_name,patch_size, data_num)

    epochs = 10
    batch_size = 1
    
    model = BayesianUNet2D((patch_size, patch_size, 3), output_channel).build()
    model.load_weights(save_path+'weights/weights.h5')
    input_shape = (patch_size,patch_size,output_channel)
    inputs2 = Input(input_shape)
    mc_samples = Lambda(lambda x: K.repeat_elements(x, mc_iteration, axis=0))(inputs2)
    logits = model(mc_samples)
    ret_shape = model.layers[-1].output_shape
    ret_shape = (-1, mc_iteration, *ret_shape[1:])
    probs = Lambda(lambda x: K.reshape(x, ret_shape))(logits)
    outputs = Lambda(lambda x: K.mean(x, axis=1))(probs)
    model2 = Model(inputs=inputs2, outputs=[outputs,outputs])

    #model2.compile(optimizer=Adam(lr=1e-6), loss=original2, metrics=['accuracy'])
    #model2.compile(optimizer=Adam(lr=1e-6), loss={'lambda_2':'categorical_crossentropy','lambda_2_1':original},loss_weights={'lambda_2':1,'lambda_2_1':8},metrics=[])
    model2.compile(optimizer=Adam(lr=1e-6), loss={'lambda_2':'categorical_crossentropy','lambda_2_1':original},metrics=[])
    
    
    train="/home/sora/new_project/crop/dataset_%d/train/"%data_num
    val="/home/sora/new_project/crop/dataset_%d/val/"%data_num
    # train -----------------------------------------------------------
    train_gen = newgenerator.ImageSequence2(train,batch_size,"train",data_num)
    valid_gen = newgenerator.ImageSequence2(val, batch_size,"val",data_num)
    os.makedirs(save_path + 'weights2', exist_ok=True)
    model_checkpoint = ModelCheckpoint(
        filepath=os.path.join(save_path,'weights2', 'weights.h5'),
        monitor="val_loss",
        verbose=1,
        save_best_only=True)
    history = model2.fit_generator(generator=train_gen,
        epochs=epochs,
        steps_per_epoch=len(train_gen),
        verbose=1,
        callbacks=[model_checkpoint],
        validation_data=valid_gen,
        validation_steps=len(valid_gen))

    tl.draw_train_loss_plot(history, save_path)
    #draw_train_loss_plot2(history, save_path)
    #draw_train_loss_plot3(history, save_path)
    #draw_train_loss_plot4(history, save_path)
    del model,model2,input_shape,outputs,probs,mc_samples,logits,ret_shape,history,train_gen,valid_gen,model_checkpoint,physical_devices,save_path2,save_path
    gc.collect()

memory growth: True
memory growth: True
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
34002/34002 [==============================] - ETA: 0s - loss: 0.1784 - lambda_2_loss: 0.1396 - lambda_2_1_loss: 0.0388

/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: Training option is ignored..
  return py_builtins.overload_of(f)(*args)



Epoch 00001: val_loss improved from inf to 0.42696, saving model to ./ips/unet_patch/unet/size_160/dataset_2/stage2/weights2/weights.h5
34002/34002 [==============================] - 3371s 99ms/step - loss: 0.1784 - lambda_2_loss: 0.1396 - lambda_2_1_loss: 0.0388 - val_loss: 0.4270 - val_lambda_2_loss: 0.3689 - val_lambda_2_1_loss: 0.0580
Epoch 2/10
34002/34002 [==============================] - ETA: 0s - loss: 0.1667 - lambda_2_loss: 0.1319 - lambda_2_1_loss: 0.0348
Epoch 00002: val_loss did not improve from 0.42696
34002/34002 [==============================] - 3346s 98ms/step - loss: 0.1667 - lambda_2_loss: 0.1319 - lambda_2_1_loss: 0.0348 - val_loss: 0.4628 - val_lambda_2_loss: 0.4065 - val_lambda_2_1_loss: 0.0562
Epoch 3/10
34002/34002 [==============================] - ETA: 0s - loss: 0.1587 - lambda_2_loss: 0.1259 - lambda_2_1_loss: 0.0328
Epoch 00003: val_loss did not improve from 0.42696
34002/34002 [==============================] - 3350s 99ms/step - loss: 0.1587 - lambda_2_

36632/36632 [==============================] - 3646s 100ms/step - loss: 0.2852 - lambda_2_loss: 0.2218 - lambda_2_1_loss: 0.0634 - val_loss: 0.2181 - val_lambda_2_loss: 0.1792 - val_lambda_2_1_loss: 0.0390
Epoch 2/10
36632/36632 [==============================] - ETA: 0s - loss: 0.2623 - lambda_2_loss: 0.2059 - lambda_2_1_loss: 0.0564
Epoch 00002: val_loss improved from 0.21814 to 0.21808, saving model to ./ips/unet_patch/unet/size_160/dataset_4/stage2/weights2/weights.h5
36632/36632 [==============================] - 3606s 98ms/step - loss: 0.2623 - lambda_2_loss: 0.2059 - lambda_2_1_loss: 0.0564 - val_loss: 0.2181 - val_lambda_2_loss: 0.1820 - val_lambda_2_1_loss: 0.0361
Epoch 3/10
36632/36632 [==============================] - ETA: 0s - loss: 0.2463 - lambda_2_loss: 0.1942 - lambda_2_1_loss: 0.0520
Epoch 00003: val_loss did not improve from 0.21808
36632/36632 [==============================] - 3595s 98ms/step - loss: 0.2463 - lambda_2_loss: 0.1942 - lambda_2_1_loss: 0.0520 - val_lo

In [8]:
#自作lossでCurriculum
for data_num in [1,2,3,4,5]:
    clear_session()
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        for k in range(len(physical_devices)):
            tf.config.experimental.set_memory_growth(physical_devices[k], True)
            print('memory growth:', tf.config.experimental.get_memory_growth(physical_devices[k]))
    else:
        print("Not enough GPU hardware devices available")
        
    save_path = tl.get_save_path(data, method, model_name,patch_size, data_num)

    epochs = 10
    batch_size = 12
    
    model = BayesianUNet2D((patch_size, patch_size, 3), output_channel).build()
    
    input_shape = (patch_size,patch_size,output_channel)
    inputs2 = Input(input_shape)
    mc_samples = Lambda(lambda x: K.repeat_elements(x, mc_iteration, axis=0))(inputs2)
    logits = model(mc_samples)
    ret_shape = model.layers[-1].output_shape
    ret_shape = (-1, mc_iteration, *ret_shape[1:])
    probs = Lambda(lambda x: K.reshape(x, ret_shape))(logits)
    outputs = Lambda(lambda x: K.mean(x, axis=1))(probs)
    model2 = Model(inputs=inputs2, outputs=[outputs,outputs])

    save_path2 = save_path######
    save_path = save_path+"stage2/"########
    model2.load_weights(save_path2+'weights2/weights.h5')
    model.compile(optimizer=Adam(lr=1e-6), loss=sensitivity_specificity, metrics=['accuracy'])
    #model2.compile(optimizer=Adam(lr=1e-6), loss={'lambda_2':'categorical_crossentropy','lambda_2_1':original},loss_weights={'lambda_2':1,'lambda_2_1':8},metrics=[])
    #model2.compile(optimizer=Adam(lr=1e-6), loss={'lambda_2':'categorical_crossentropy','lambda_2_1':original},metrics=[])
    
    
    
    
    train="/home/sora/new_project/crop/dataset_%d/train/"%data_num
    val="/home/sora/new_project/crop/dataset_%d/val/"%data_num
    # train -----------------------------------------------------------
    train_gen = newgenerator.ImageSequence(train,batch_size,"train",data_num)
    valid_gen = newgenerator.ImageSequence(val, batch_size,"val",data_num)
    os.makedirs(save_path + 'weights', exist_ok=True)
    model_checkpoint = ModelCheckpoint(
        filepath=os.path.join(save_path,'weights', 'weights.h5'),
        monitor="val_loss",
        verbose=1,
        save_best_only=True)
    history = model.fit_generator(generator=train_gen,
        epochs=epochs,
        steps_per_epoch=len(train_gen),
        verbose=1,
        callbacks=[model_checkpoint],
        validation_data=valid_gen,
        validation_steps=len(valid_gen))

    tl.draw_train_loss_plot(history, save_path)
    #draw_train_loss_plot2(history, save_path)
    #draw_train_loss_plot3(history, save_path)
    #draw_train_loss_plot4(history, save_path)
    #del model,model2,input_shape,outputs,probs,mc_samples,logits,ret_shape,history,train_gen,valid_gen,model_checkpoint,physical_devices,save_path2,save_path
    del model,history,train_gen,valid_gen,model_checkpoint,physical_devices,save_path2,save_path    
    gc.collect()

memory growth: True
memory growth: True
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
2636/2636 [==============================] - ETA: 0s - loss: 0.0317 - accuracy: 0.9500

/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: Training option is ignored..
  return py_builtins.overload_of(f)(*args)



Epoch 00001: val_loss improved from inf to 0.05465, saving model to ./ips/unet_patch/unet/size_160/dataset_1/stage2/weights/weights.h5
2636/2636 [==============================] - 925s 351ms/step - loss: 0.0317 - accuracy: 0.9500 - val_loss: 0.0547 - val_accuracy: 0.9098
Epoch 2/10
2636/2636 [==============================] - ETA: 0s - loss: 0.0290 - accuracy: 0.9508
Epoch 00002: val_loss improved from 0.05465 to 0.05292, saving model to ./ips/unet_patch/unet/size_160/dataset_1/stage2/weights/weights.h5
2636/2636 [==============================] - 306s 116ms/step - loss: 0.0290 - accuracy: 0.9508 - val_loss: 0.0529 - val_accuracy: 0.9090
Epoch 3/10
2636/2636 [==============================] - ETA: 0s - loss: 0.0280 - accuracy: 0.9513
Epoch 00003: val_loss improved from 0.05292 to 0.05215, saving model to ./ips/unet_patch/unet/size_160/dataset_1/stage2/weights/weights.h5
2636/2636 [==============================] - 306s 116ms/step - loss: 0.0280 - accuracy: 0.9513 - val_loss: 0.0522 - 

Epoch 6/10
2918/2918 [==============================] - ETA: 0s - loss: 0.0342 - accuracy: 0.9380
Epoch 00006: val_loss improved from 0.04083 to 0.04076, saving model to ./ips/unet_patch/unet/size_160/dataset_3/stage2/weights/weights.h5
2918/2918 [==============================] - 339s 116ms/step - loss: 0.0342 - accuracy: 0.9380 - val_loss: 0.0408 - val_accuracy: 0.9273
Epoch 7/10
2918/2918 [==============================] - ETA: 0s - loss: 0.0339 - accuracy: 0.9385
Epoch 00007: val_loss did not improve from 0.04076
2918/2918 [==============================] - 337s 116ms/step - loss: 0.0339 - accuracy: 0.9385 - val_loss: 0.0412 - val_accuracy: 0.9263
Epoch 8/10
2918/2918 [==============================] - ETA: 0s - loss: 0.0334 - accuracy: 0.9395
Epoch 00008: val_loss improved from 0.04076 to 0.04067, saving model to ./ips/unet_patch/unet/size_160/dataset_3/stage2/weights/weights.h5
2918/2918 [==============================] - 339s 116ms/step - loss: 0.0334 - accuracy: 0.9395 - val_lo

In [5]:
for mode in ["test","train"] :
    for data_num in [1,2,3,4,5]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        os.makedirs(save_path + '%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"%s"%mode, exist_ok=True)
        os.makedirs(save_path+"label", exist_ok=True)
        os.makedirs(save_path + '%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/probability'%mode, exist_ok=True)

In [5]:
for mode in ["test"] :
    for data_num in [1]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        os.makedirs(save_path + 'stage2/%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"stage2/%s"%mode, exist_ok=True)
        os.makedirs(save_path+"stage2/label", exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/probability'%mode, exist_ok=True)

In [8]:
#予測マップ、uncertainty map作成
height=1200
width=1600
step=10
for data_num in [2,3,4,5]:
    clear_session()
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    save_path = save_path
    
    model = BayesianUNet2D((patch_size, patch_size, 3), 3).build()
    input_shape = (patch_size,patch_size,output_channel)
    inputs2 = Input(input_shape)
    mc_samples = Lambda(lambda x: K.repeat_elements(x, mc_iteration, axis=0))(inputs2)
    logits = model(mc_samples)
    ret_shape = model.layers[-1].output_shape
    ret_shape = (-1, mc_iteration, *ret_shape[1:])
    probs = Lambda(lambda x: K.reshape(x, ret_shape))(logits)
    outputs = Lambda(lambda x: K.mean(x, axis=1))(probs)
    model2 = Model(inputs=inputs2, outputs=[outputs,outputs])
    
    save_path = save_path+"stage2/"
    model2.load_weights(save_path+'weights2/weights.h5')
    
    for mode in ["test"]:
        img_path, mask_path = tl.data_path_load(data,mode,data_num)
        for i_path in img_path:
            print("************************************")
            print(i_path)
            img = Image.open(i_path)
            img = np.array(img)
            name = os.path.basename(i_path)[:-4]
            Yp=[]
            y=0
            while y < (height - patch_size + 1):
                x = 0
                while x < (1600 - patch_size + 1):
                    crop_img = img[y:patch_size+y, x:patch_size+x]
                    crop_img = crop_img/255.
                    
                    tmp,_ = model2.predict(crop_img[np.newaxis, :, :, :])
                    tmp = np.reshape(tmp, (patch_size,patch_size,n_class))
                    Yp.append(tmp)
                    del crop_img,tmp,_
                    gc.collect()
                    x+=step   
                y+=step
            Yp = np.array(Yp)
            
            print("Predict finish")
            vis_img, label_img,e,pro = make_prediction_map(patch_size,
                                                             step,
                                                             width,
                                                             height,
                                                             Yp,
                                                             reflect,
                                                             data,
                                                             n_class)
            vis_img = vis_img[:, :, [2, 1, 0]]
            pro = pro[:, :, [2, 1, 0]]
            cv2.imwrite(save_path + '%s/' % mode+name + '.png', vis_img)
            cv2.imwrite(save_path + '%s_un/probability/' % mode+name + '.png', pro)
            if mode == "test":
                convert2label(name, save_path)
            tmp=e
            e_1 = e*255
            e_1 = Image.fromarray(e_1)
            e_1 = e_1.convert('RGB')
            e_1.save(save_path + '%s_un/uncertainty/'%mode + name + '.png')
            e[e>=0.5]=255
            e[e<0.5]=0
            e=Image.fromarray(e)
            e = e.convert('RGB')
            e.save(save_path + '%s_un/uncertainty_T_0.5/'%mode + name + '.png')
            del Yp,vis_img, label_img,e,e_1
            gc.collect()
            print("************************************")
    del model,model2,input_shape,outputs,probs,mc_samples,logits,ret_shape
    gc.collect()

************************************
data/ips/img/E3230662.JPG


/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: Training option is ignored..
  return py_builtins.overload_of(f)(*args)


Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230715.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230729.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230705.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230734.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230709.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230730.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230667.JPG
Predict finish
Start

In [7]:
def pre_data_path_load(path):
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

In [8]:
#correctness map作成
for data_num in [2,3,4,5]:
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    mode="test"
    model_name="unet"
    size=160
    #path=("./%s/%s/%s/size_%d/dataset_%d/" %(data, method,model_name,size,data_num))
    path=("./%s/%s/%s/size_%d/dataset_%d/stage2/" %(data, method,model_name,size,data_num))
    #path=("./%s/%s/%s/size_%d/dataset_%d/stage2/stage3/stage4/" %(data, method,model_name,size,data_num))

    pre_path = pre_data_path_load(path+"%s/"%mode)

    mask_path.sort()
    pre_path.sort()
    for m_path,p_path in zip(mask_path,pre_path):
        print(m_path)
        name = os.path.basename(m_path)[:-4]
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
        m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
        m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
        m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]

        mask = get_ytrue(m_mask, 'ips')
        oor = ~(mask == 0) *1

        p_image = cv2.imread(p_path)
        p_image = cv2.cvtColor(p_image, cv2.COLOR_BGR2RGB)
        p_image[:,:,0] = p_image[:,:,0]* oor 
        p_image[:,:,1] = p_image[:,:,1] * oor 
        p_image[:,:,2] = p_image[:,:,2] * oor
        pre = get_ytrue(p_image, 'ips')
        true=mask-pre
        true[true!=0]=255
        true=true*oor
        true=Image.fromarray(true)
        true = true.convert('RGB')
        true.save(path + 'test_un/correctness/' + name + '.png')

data/ips/mask/E3230662.png
data/ips/mask/E3230665.png
data/ips/mask/E3230667.png
data/ips/mask/E3230668.png
data/ips/mask/E3230678.png
data/ips/mask/E3230692.png
data/ips/mask/E3230705.png
data/ips/mask/E3230709.png
data/ips/mask/E3230715.png
data/ips/mask/E3230729.png
data/ips/mask/E3230730.png
data/ips/mask/E3230734.png
data/ips/mask/E3230652.png
data/ips/mask/E3230666.png
data/ips/mask/E3230679.png
data/ips/mask/E3230685.png
data/ips/mask/E3230688.png
data/ips/mask/E3230693.png
data/ips/mask/E3230698.png
data/ips/mask/E3230704.png
data/ips/mask/E3230707.png
data/ips/mask/E3230708.png
data/ips/mask/E3230719.png
data/ips/mask/E3230656.png
data/ips/mask/E3230661.png
data/ips/mask/E3230669.png
data/ips/mask/E3230681.png
data/ips/mask/E3230695.png
data/ips/mask/E3230696.png
data/ips/mask/E3230699.png
data/ips/mask/E3230713.png
data/ips/mask/E3230716.png
data/ips/mask/E3230722.png
data/ips/mask/E3230727.png
data/ips/mask/E3230733.png
data/ips/mask/E3230657.png
data/ips/mask/E3230659.png
d